In [3]:
import pandas as pd

data = pd.read_csv('data/players.csv',delimiter=';',encoding='latin-1',index_col=0)

#data.head(10)

data.insert(0,'ID',data.index)

players_C = data[data.Pos == "C"]
players_SG = data[data.Pos == "SG"]
players_PG = data[data.Pos == "PG"]
players_PF = data[data.Pos == 'PF']
players_SF = data[data.Pos == 'SF']

data.drop(['ID'], axis=1)

data.head(10)


,ID,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
Rk,,,,,,,,,,,,,,,,,,,,,
1,1,Precious Achiuwa,C,23,TOR,55,12,20.7,3.6,7.3,...,0.702,1.8,4.1,6.0,0.9,0.6,0.5,1.1,1.9,9.2
2,2,Steven Adams,C,29,MEM,42,42,27.0,3.7,6.3,...,0.364,5.1,6.5,11.5,2.3,0.9,1.1,1.9,2.3,8.6
3,3,Bam Adebayo,C,25,MIA,75,75,34.6,8.0,14.9,...,0.806,2.5,6.7,9.2,3.2,1.2,0.8,2.5,2.8,20.4
4,4,Ochai Agbaji,SG,22,UTA,59,22,20.5,2.8,6.5,...,0.812,0.7,1.3,2.1,1.1,0.3,0.3,0.7,1.7,7.9
5,5,Santi Aldama,PF,22,MEM,77,20,21.8,3.2,6.8,...,0.750,1.1,3.7,4.8,1.3,0.6,0.6,0.8,1.9,9.0
6,6,Nickeil Alexander-Walker,SG,24,TOT,59,3,15.0,2.2,5.0,...,0.667,0.3,1.5,1.7,1.8,0.5,0.4,0.9,1.5,6.2
6,6,Nickeil Alexander-Walker,SG,24,UTA,36,3,14.7,2.3,4.7,...,0.692,0.2,1.4,1.6,2.1,0.7,0.4,1.3,1.6,6.3
6,6,Nickeil Alexander-Walker,SG,24,MIN,23,0,15.5,2.1,5.4,...,0.619,0.3,1.5,1.8,1.4,0.3,0.3,0.4,1.3,5.9
7,7,Grayson Allen,SG,27,MIL,72,70,27.4,3.4,7.7,...,0.905,0.8,2.4,3.3,2.3,0.9,0.2,1.0,1.6,10.4


In [ ]:
# DATA LOADER
import io
import pandas as pd
import requests
if 'data_loader' not in globals():
    from mage_ai.data_preparation.decorators import data_loader
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@data_loader
def load_data(*args, **kwargs):
    """
    Template code for loading data from any source.
    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    # Specify your data loading logic here
    url='https://storage.googleapis.com/nba-project-natevu/players.csv'
    response = requests.get(url)

    return pd.read_csv(io.StringIO(response.text),sep =';')


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
    

#DATA TRANSFORMER
import pandas as pd

if 'transformer' not in globals():
    from mage_ai.data_preparation.decorators import transformer
if 'test' not in globals():
    from mage_ai.data_preparation.decorators import test


@transformer
def transform(data, *args, **kwargs):
    """
    Template code for a transformer block.

    Add more parameters to this function if this block has multiple parent blocks.
    There should be one parameter for each output variable from each parent block.

    Args:
        data: The output from the upstream parent block
        args: The output from any additional upstream blocks (if applicable)

    Returns:
        Anything (e.g. data frame, dictionary, array, int, str, etc.)
    """
    # Creating metrics
    EfficiencyC = round(2*data.TRB + 1.5*data.BLK + 1.25*data.PTS + data.AST + data.STL - data.TOV - (data.FGA - data.FG) - (data.FTA - data.FT) - data.PF,2)
    EfficiencySG = round(data.TRB + data.BLK + 1.75*data.PTS + data.AST + 1.2*data.STL - data.TOV - (data.FGA - data.FG) - (data.FTA - data.FT) - data.PF,2)
    EfficiencySF = round(1.5*data.TRB + data.BLK + 1.5*data.PTS + data.AST + data.STL - data.TOV - (data.FGA - data.FG) - (data.FTA - data.FT) - data.PF,2)
    EfficiencyPG = round(data.TRB + data.BLK + 1.25*data.PTS + 2*data.AST + 1.2*data.STL - data.TOV - (data.FGA - data.FG) - (data.FTA - data.FT) - data.PF,2)
    EfficiencyPF = round(1.25*data.TRB + 1.5*data.BLK + 1.5*data.PTS + data.AST + data.STL - data.TOV - (data.FGA - data.FG) - (data.FTA - data.FT) - data.PF,2)

    data.insert(29,'EFFc',EfficiencyC)
    data.insert(30,'EFFsg',EfficiencySG)
    data.insert(31,'EFFsf',EfficiencySF)
    data.insert(32,'EFFpg',EfficiencyPG)
    data.insert(33,'EFFpf',EfficiencyPF)

    #Dividing datasets
    #players_C=data[data.Pos == 'C']
    #players_SG=data[data.Pos == 'SG']
    #players_SF=data[data.Pos == 'SF']
    #players_PG=data[data.Pos == 'PG']
    #players_PF=data[data.Pos == 'PF']
    
    return data
    
    #return {"players_C":players_C.to_dict(orient="dict"),
        #"players_SG":players_SG.to_dict(orient="dict"),
        #"players_SF":players_SF.to_dict(orient="dict"),
        #"players_PG":players_PG.to_dict(orient="dict"),
        #"players_PF":players_PF.to_dict(orient="dict")
    #}


@test
def test_output(output, *args) -> None:
    """
    Template code for testing the output of the block.
    """
    assert output is not None, 'The output is undefined'
    

#DATA EXPORTER
from mage_ai.settings.repo import get_repo_path
from mage_ai.io.bigquery import BigQuery
from mage_ai.io.config import ConfigFileLoader
from pandas import DataFrame
from os import path

if 'data_exporter' not in globals():
    from mage_ai.data_preparation.decorators import data_exporter


@data_exporter
def export_data_to_big_query(data, **kwargs) -> None:
    """
    Template for exporting data to a BigQuery warehouse.
    Specify your configuration settings in 'io_config.yaml'.

    Docs: https://docs.mage.ai/design/data-loading#bigquery
    """
    table_id = 'hardy-answer-335705.nba_project.players'
    #table_id = 'hardy-answer-335705.nba_project.players_C'
    #table_id = 'hardy-answer-335705.nba_project.players_PF'
    #table_id = 'hardy-answer-335705.nba_project.players_PG'
    #table_id = 'hardy-answer-335705.nba_project.players_SG'
    #table_id = 'hardy-answer-335705.nba_project.players_SF'
    
    config_path = path.join(get_repo_path(), 'io_config.yaml')
    config_profile = 'default'

    BigQuery.with_config(ConfigFileLoader(config_path, config_profile)).export(
        data
        #DataFrame = data['players_C']
        #DataFrame = data['players_PF']
        #DataFrame = data['players_PG']
        #DataFrame = data['players_SG']
        #DataFrame = data['players_SF']
        table_id,
        if_exists='replace',  # Specify resolution policy if table name already exists
    )
    
